<a href="https://colab.research.google.com/github/nisha1365/Sanofi-POC/blob/main/BioBert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:


df = pd.read_csv("/content/my_data.csv")
df

,Unnamed: 0,tweet_id,external_twitter_id,text,author_id,created_at,lang,referenced_tweets_type,referenced_tweets_id,infos,association_tweet_id,association_scope_id
0,0,1030258,1.684840e+18,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,9.050890e+17,28-07-2023 08:18,en,0,0.000000e+00,{},1030258,10
1,1,1030186,1.684590e+18,"Roche Drops Hemophilia A Candidate, Prepares f...",7.638583e+07,27-07-2023 15:51,en,0,0.000000e+00,{},1030186,10
2,2,1030263,1.684810e+18,Roche moves one Spark hemophilia gene therapy ...,1.481280e+18,28-07-2023 06:34,en,0,0.000000e+00,{},1030263,10
3,3,1030274,1.684720e+18,On #WorldHepatitisDay we put middle finger up ...,1.145270e+18,28-07-2023 00:27,en,0,0.000000e+00,{},1030274,10
4,4,1030288,1.684660e+18,"One of our newest internship programs, launche...",2.112887e+08,27-07-2023 20:13,en,3,1.684630e+18,{},1030288,10
...,...,...,...,...,...,...,...,...,...,...,...,...
29798,29798,1683839551645814784,NaN,"Hear more from Mosi Williams, Pys.D, on the la...",1.479181e+18,2023-07-25 14:00:07,en,0,0.000000e+00,{},1683839551645814784,10
29799,29799,1683938283003183104,NaN,"Was discussing hemophilia with an intelligent,...",2.760856e+08,2023-07-25 20:32:27,en,0,0.000000e+00,{},1683938283003183104,10
29800,29800,1683861079674195968,NaN,The affected &amp; infected have waited decade...,1.104181e+08,2023-07-25 15:25:40,en,3,1.683849e+18,{},1683861079674195968,10
29801,29801,1683918306082955264,NaN,"@MohmdBio Sickle cell anemia, Thalassemia, Hem...",1.179873e+18,2023-07-25 19:13:04,en,2,1.683918e+18,{},1683918306082955264,10


In [3]:

columns_to_keep = ['author_id','tweet_id','text']

new_df = df[columns_to_keep]
new_df.head()

,author_id,tweet_id,text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...
4,2.112887e+08,1030288,"One of our newest internship programs, launche..."


In [4]:

!pip install tweet-preprocessor

In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# import tweet preprocessor
import preprocessor as p

import nltk
import gensim
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [6]:

# Customizing the preprocessor to exclude removing hashtags since they are valuable as a rich information
p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION,p.OPT.SMILEY,p.OPT.NUMBER )

#forming a separate feature for cleaned tweets
for i,v in enumerate(new_df['text']):
    new_df.loc[i,'new_text'] = p.clean(v)

new_df.head()

<ipython-input-6-97c3e15aa739>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[i,'new_text'] = p.clean(v)


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks Challenge #AxioActive On the th of Augus...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","Roche Drops Hemophilia A Candidate, Prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,Roche moves one Spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,On #WorldHepatitisDay we put middle finger up ...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","One of our newest internship programs, launche..."


In [7]:
#REMOVE ONLY THE '#'NOT THE WORD AFTER
def remove_hashtag_sign(text):
    text = re.sub(r'#', '', text)
    return text

new_df['new_text'] = new_df['new_text'].apply(lambda x:remove_hashtag_sign(x))

<ipython-input-7-95cc9cf31c2a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_text'] = new_df['new_text'].apply(lambda x:remove_hashtag_sign(x))


In [8]:

#Remove extra white spaces, punctuation and apply lower casing
new_df['new_text'] = new_df['new_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
new_df.head()

<ipython-input-8-e53ee1d17d90>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_text'] = new_df['new_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","roche drops hemophilia a candidate, prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","one of our newest internship programs, launche..."


In [9]:
new_df


,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","roche drops hemophilia a candidate, prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","one of our newest internship programs, launche..."
...,...,...,...,...
29798,1.479181e+18,1683839551645814784,"Hear more from Mosi Williams, Pys.D, on the la...","hear more from mosi williams, pys.d, on the la..."
29799,2.760856e+08,1683938283003183104,"Was discussing hemophilia with an intelligent,...","was discussing hemophilia with an intelligent,..."
29800,1.104181e+08,1683861079674195968,The affected &amp; infected have waited decade...,the affected &amp; infected have waited decade...
29801,1.179873e+18,1683918306082955264,"@MohmdBio Sickle cell anemia, Thalassemia, Hem...","sickle cell anemia, thalassemia, hemophilia"


In [10]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from gensim import corpora
from gensim.models import LdaModel

In [11]:
new_df

,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","roche drops hemophilia a candidate, prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","one of our newest internship programs, launche..."
...,...,...,...,...
29798,1.479181e+18,1683839551645814784,"Hear more from Mosi Williams, Pys.D, on the la...","hear more from mosi williams, pys.d, on the la..."
29799,2.760856e+08,1683938283003183104,"Was discussing hemophilia with an intelligent,...","was discussing hemophilia with an intelligent,..."
29800,1.104181e+08,1683861079674195968,The affected &amp; infected have waited decade...,the affected &amp; infected have waited decade...
29801,1.179873e+18,1683918306082955264,"@MohmdBio Sickle cell anemia, Thalassemia, Hem...","sickle cell anemia, thalassemia, hemophilia"


In [12]:
new_df = new_df.head(1000)
new_df

,author_id,tweet_id,text,new_text
0,9.050890e+17,1030258,3 peaks Challenge #AxioActive💪\n\nOn the 12th ...,peaks challenge axioactive on the th of august...
1,7.638583e+07,1030186,"Roche Drops Hemophilia A Candidate, Prepares f...","roche drops hemophilia a candidate, prepares f..."
2,1.481280e+18,1030263,Roche moves one Spark hemophilia gene therapy ...,roche moves one spark hemophilia gene therapy ...
3,1.145270e+18,1030274,On #WorldHepatitisDay we put middle finger up ...,on worldhepatitisday we put middle finger up t...
4,2.112887e+08,1030288,"One of our newest internship programs, launche...","one of our newest internship programs, launche..."
...,...,...,...,...
995,1.935531e+08,1033001,IU researchers uncover mysteries behind immune...,iu researchers uncover mysteries behind immune...
996,1.409967e+08,1033007,@HemoIX @JonRob08 @ProfMakris @NHF_Hemophilia ...,im a patient
997,1.498170e+18,1032973,IU researchers uncover mysteries behind immune...,iu researchers uncover mysteries behind immune...
998,2.280750e+09,1032974,"@MrDelicious13 @caseyboguslaw Jesus, we’ve got...","jesus, weve got that back woods, swamp dna, th..."


In [13]:
# Assuming 'new_df' is your DataFrame and 'new_text' is the column containing cleaned text
data = new_df['new_text'].values.tolist()

In [14]:
data

['peaks challenge axioactive on the th of august , some of the axiologik teammates will be taking on the yorkshire peaks within hours to raise significant funds for the haemophilia society. all donations are truly appreciated:',
 'roche drops hemophilia a candidate, prepares for the impacts of ira',
 'roche moves one spark hemophilia gene therapy into phiii, cuts another',
 'on worldhepatitisday we put middle finger up to australias corrupt health officials whove let thousands die due to their coverup of the infectedbloodscandal which has only been possible through complicit journalists who support genocide. haemophilia contaminatedblood',
 'one of our newest internship programs, launched in ! raquel is researching how high-cost hemophilia therapies can be optimally managed through specialtypharmacy. nationalinternday amcpfdn',
 'delighted and honored to see our paper published with a commentary by dr. kate pratt and featured in the latest july issue of hemophilia haemophilia',
 'learn

In [15]:
!pip install pyLDAvis

In [16]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim

In [17]:
# Load BioBERT model and tokenizer from Hugging Face
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
# Tokenize and encode the text data
encoded_data = tokenizer(data, padding=True, truncation=True, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
# Obtain the sentence embeddings
with torch.no_grad():
    model_output = model(**encoded_data)

# Convert the model output to numpy array
sentence_embeddings = model_output.last_hidden_state.mean(dim=1).numpy()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Iterate over different numbers of topics and select the best one based on coherence score
best_coherence = -1
best_lda_model = None
best_num_topics = 0

for num_topics in range(3, 10):  # Try different numbers of topics
    texts = [text.split() for text in data]  # Pre-process the text data
    id2word = corpora.Dictionary(texts)
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)

    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()

    # Select the best model based on coherence score
    if coherence > best_coherence:
        best_coherence = coherence
        best_lda_model = lda_model
        best_num_topics = num_topics

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# Print the best coherence score and number of topics
print(f"The best coherence score: {best_coherence} with {best_num_topics} topics")

The best coherence score: 0.3750760516496162 with 8 topics


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
vis = pyLDAvis.gensim.prepare(best_lda_model, corpus, id2word)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
